In [1]:
import pandas as pd

In [2]:
# Labels from https://www.nature.com/articles/s41586-021-03493-4 Stukalov et al. 2021
# Direct download: https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-021-03493-4/MediaObjects/41586_2021_3493_MOESM6_ESM.xlsx
df = pd.read_excel('41586_2021_3493_MOESM6_ESM.xlsx', sheet_name='A - Significant interactions')
df

/Users/yang.an/mamba/envs/tf2/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/yang.an/mamba/envs/tf2/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,bait_organism,bait_name,gene_name,protein_description,majority_protein_acs,fold_change_log2,p_value,fold change_threshold,comparison_batch,fold_change_log2_batch,...,oeproteome_p_value,oeproteome_is_hit,rnaseq_comparison,rnaseq_is_hit,proteome_comparison,proteome_is_hit,phospho_comparison,phospho_is_hit,ubi_comparison,ubi_is_hit
0,SARS-CoV,E,UFD1,Ubiquitin recognition factor in ER-associated ...,Q92890;Q92890-1;Q92890-3,9.969620,6.016913e-276,2,SARS_CoV_E_vs_B5_others,10.077186,...,0.445602,0.0,NaN,0.0,NaN,0.0,S247@UFD1[24h:shared(-) 36h:shared(-)] S299@UF...,1.0,NaN,0.0
1,SARS-CoV,E,NPLOC4,Nuclear protein localization protein 4 homolog...,Q8TAT6;Q8TAT6-2,8.854112,3.193217e-20,2,SARS_CoV_E_vs_B5_others,9.110157,...,0.009037,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0
2,SARS-CoV,E,SARS_CoV_E,NO DESCRIPTION,P59637,9.536133,1.491272e-68,2,SARS_CoV_E_vs_B5_others,9.622996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K63@SARS_CoV_E[12h:SARS_CoV(+) 24h:SARS_CoV(+)],1.0
3,SARS-CoV,E,RAB34,"Ras-related protein Rab-34, isoform NARR",P0DI83,7.876418,4.422959e-11,2,SARS_CoV_E_vs_B5_others,8.156219,...,0.052086,0.0,NaN,0.0,NaN,0.0,S123@RAB34[36h:shared(-)],1.0,NaN,NaN
4,SARS-CoV,E,FAM241B,Protein FAM241B...,Q96D05;Q96D05-2,6.863366,3.467090e-04,2,SARS_CoV_E_vs_B5_others,6.953465,...,0.096263,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,SARS-CoV,S,ZDHHC5,Palmitoyltransferase ZDHHC5...,Q9C0B5;Q9C0B5-2,7.105674,3.793086e-04,2,SARS_CoV_S_vs_B4_others,7.465508,...,0.760743,0.0,NaN,0.0,NaN,0.0,T348@ZDHHC5[12h:shared(+) 24h:SARS_CoV2(+) 36h...,1.0,NaN,0.0
2102,SARS-CoV,S,SARS_CoV_S,Spike glycoprotein,P59594,16.554913,0.000000e+00,2,SARS_CoV_S_vs_B4_others,16.492960,...,NaN,NaN,NaN,NaN,NaN,0.0,T215@SARS_CoV_S[36h:SARS_CoV(+)],1.0,K188@SARS_CoV_S[12h:SARS_CoV(+) 24h:SARS_CoV(+...,1.0
2103,SARS-CoV-2,S,ZDHHC5,Palmitoyltransferase ZDHHC5...,Q9C0B5;Q9C0B5-2,7.446649,7.353729e-06,2,SARS_CoV2_S_vs_B5_others,7.662380,...,0.565619,0.0,NaN,0.0,NaN,0.0,T348@ZDHHC5[12h:shared(+) 24h:SARS_CoV2(+) 36h...,1.0,NaN,0.0
2104,SARS-CoV-2,S,GOLGA7,Golgin subfamily A member 7...,Q7Z5G4;Q7Z5G4-3,7.566474,1.689030e-04,2,SARS_CoV2_S_vs_B5_others,7.732983,...,0.218128,0.0,NaN,0.0,NaN,0.0,NaN,NaN,K95@GOLGA7[24h:shared(+)],1.0


In [3]:
# Extract SARS-CoV host factors
df_sars = df[~df['gene_name'].str.contains('SARS')]
df_sars = df_sars[df_sars['bait_organism'] == 'SARS-CoV']['majority_protein_acs'].str.split(';').explode()
df_sars[~df_sars.str.contains('-') & ~df_sars.str.contains('SARS')].nunique()

612

In [4]:
predictions = pd.read_csv('../benchmark/prediction_values_all_models.csv.zip')

In [5]:
predictions['SARS-CoV'] = predictions['protein_ac'].isin(df[df['bait_organism'] == 'SARS-CoV']['majority_protein_acs'].str.split(';').explode().tolist())

In [6]:
# ideal threshold from SARS-CoV-2
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, accuracy_score, precision_score, average_precision_score, recall_score, precision_recall_curve
import numpy as np

modality = 'tf_finetuned'
config = 'tf_finetuned_ensemble'
Ks = [50, 100, 200]
results = []

cov = 'SARS-CoV'
# Best threshold based on SARS-CoV-2
val_subset = predictions[predictions[f'group_split_0'].isin(['train', 'val'])].copy()

precision, recall, thresholds = precision_recall_curve(val_subset['label'] == '1.0', val_subset[f'{modality}_ensemble'])
f1_scores = 2*precision*recall / (precision+recall)
best_threshold = thresholds[f1_scores.argmax()]

test_subset = predictions[predictions['group_split_0'] == 'test'].copy()
test_subset = predictions.copy()

result = {}
result.update({
          'Virus': cov,
          f'AUROC': roc_auc_score(test_subset[cov], test_subset[config]),
          f'APS': average_precision_score(test_subset[cov], test_subset[config]),
          f'F1-score': f1_score(test_subset[cov], test_subset[config] > best_threshold, ),
          f'Precision': precision_score(test_subset[cov], test_subset[config] > best_threshold, ),
          f'Recall': recall_score(test_subset[cov], test_subset[config] > best_threshold, ),
         })

for K in Ks:
    test_subset_top_k = test_subset.loc[test_subset[config].nlargest(K).index]
    result.update({f'Precision@{K}': precision_score(test_subset_top_k[cov], np.ones_like(test_subset_top_k[config]))})


results.append(result)

results = pd.DataFrame(results)
results.round(2)

,Virus,AUROC,APS,F1-score,Precision,Recall,Precision@50,Precision@100,Precision@200
0,SARS-CoV,0.8,0.11,0.19,0.12,0.42,0.16,0.19,0.2


# HIV

In [7]:
# Labels Supp Table 1 from https://journals.asm.org/doi/10.1128/mbio.00009-23 Montoya et al. 2023
# Direct download: https://journals.asm.org/doi/suppl/10.1128/mbio.00009-23/suppl_file/mbio.00009-23-st001.xlsx
# The file itself is corrupted and cannot be loaded directly in pandas. Please open using Excel and save the first sheet as csv

df_hiv = pd.read_csv('mbio.00009-23-st001.csv')
df_hiv = df_hiv[~df_hiv['gene id'].str.contains('synNTC')]
df_hiv = df_hiv.iloc()[:368]  # Number used in Montoya et al. 2023
df_hiv

,gene id,number of guides,neg|MAGeCK score,neg|p-value,neg|fdr (false discovery rate),neg|rank,neg|lfc (log fold change),pos|MAGeCK score,pos|p-value,pos|fdr,pos|rank,pos|lfc
0,UBE2M,8,9.850000e-10,0.000005,0.000381,1,-1.623100,1.000000,1.000000,0.999995,1052,-1.623100
1,CCNT1,8,1.430000e-09,0.000005,0.000381,2,-2.613600,1.000000,1.000000,0.999995,1054,-2.613600
2,PPP2CA,8,1.010000e-08,0.000005,0.000381,3,-0.859250,1.000000,1.000000,0.999995,1053,-0.859250
3,ZBTB7A,8,1.610000e-08,0.000005,0.000381,4,-1.503500,1.000000,1.000000,0.999995,1051,-1.503500
4,TRNAU1AP,8,1.010000e-07,0.000005,0.000381,5,-0.525600,1.000000,1.000000,0.999995,1049,-0.525600
...,...,...,...,...,...,...,...,...,...,...,...,...
481,SHISAL2A,8,3.108300e-01,0.596350,0.999995,482,-0.027965,0.280210,0.570040,0.999995,593,-0.027965
485,RNF17,6,3.184000e-01,0.565400,0.999995,486,-0.211300,0.560600,0.774400,0.999995,774,-0.211300
488,ELOVL1,8,3.207900e-01,0.605520,0.999995,489,0.069355,0.411670,0.695810,0.999995,678,0.069355
489,DYRK1A,8,3.208100e-01,0.605520,0.999995,490,-0.008780,0.005253,0.027557,0.543204,49,-0.008780


In [8]:
# ideal threshold from SARS-CoV-2
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, accuracy_score, precision_score, average_precision_score, recall_score, precision_recall_curve
import numpy as np


predictions['HIV'] = predictions['gene'].isin(df_hiv['gene id'])

modality = 'tf_finetuned'
config = 'tf_finetuned_ensemble'
Ks = [50, 100, 200]
results = []

for cov in ['SARS-CoV', 'HIV']:
    # Best threshold based on SARS-CoV-2
    val_subset = predictions[predictions[f'group_split_0'].isin(['train', 'val'])].copy()
    
    precision, recall, thresholds = precision_recall_curve(val_subset['label'] == '1.0', val_subset[f'{modality}_ensemble'])
    f1_scores = 2*precision*recall / (precision+recall)
    best_threshold = thresholds[f1_scores.argmax()]
    
    test_subset = predictions[predictions['group_split_0'] == 'test'].copy()
    test_subset = predictions.copy()
    # test_subset = predictions[predictions['gene'].isin(df_hiv2['gene id'])].copy()
    
    result = {}
    result.update({
              'Virus': cov,
              f'AUROC': roc_auc_score(test_subset[cov], test_subset[config]),
              f'APS': average_precision_score(test_subset[cov], test_subset[config]),
              f'F1-score': f1_score(test_subset[cov], test_subset[config] > best_threshold, ),
              f'Precision': precision_score(test_subset[cov], test_subset[config] > best_threshold, ),
              f'Recall': recall_score(test_subset[cov], test_subset[config] > best_threshold, ),
             })

    for K in Ks:
        test_subset_top_k = test_subset.loc[test_subset[config].nlargest(K).index]
        result.update({f'Precision@{K}': precision_score(test_subset_top_k[cov], np.ones_like(test_subset_top_k[config]))})


    results.append(result)

results = pd.DataFrame(results)
results.round(2)

,Virus,AUROC,APS,F1-score,Precision,Recall,Precision@50,Precision@100,Precision@200
0,SARS-CoV,0.80,0.11,0.19,0.12,0.42,0.16,0.19,0.20
1,HIV,0.64,0.03,0.05,0.03,0.18,0.06,0.05,0.03
